<a href="https://colab.research.google.com/github/Rajarshi12321/gpt-dev-token-former-/blob/main/gpt_dev_(token_former).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Building a GPT

Companion notebook to the [Zero To Hero](https://karpathy.ai/zero-to-hero.html) video on GPT.

In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
!wget https://raw.githubusercontent.com/Rajarshi12321/gpt-dev-token-former-/refs/heads/main/pokemon.txt

--2024-11-28 15:07:18--  https://raw.githubusercontent.com/Rajarshi12321/gpt-dev-token-former-/refs/heads/main/pokemon.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 509985 (498K) [text/plain]
Saving to: ‘pokemon.txt’

pokemon.txt         100%[===================>] 498.03K  --.-KB/s    in 0.05s   

2024-11-28 15:07:19 (10.1 MB/s) - ‘pokemon.txt’ saved [509985/509985]



In [ ]:
# read it in to inspect it
with open('pokemon.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  509245


In [ ]:
# let's look at the first 1000 characters
print(text[:1000])


---------------------Chaper - 1--------------------

Ash Ketchum: Being More
VERY IMPORTANT NOTICE: This is a repost of a story that I deleted quite some time ago, as I lost interest in it, but as I am sometimes still receiving PMs telling me that people miss it, I am reposting it for those who are still interested in reading it and for posterity's sake. Be warned, though. I have no intention to work on this story any time soon; only if and when my interest for this story returns will I work on it, and that is a very big if.

I will post the other finished chapters of this story in the coming week. I noticed that the grammer isn't up to my standards anymore as I have progressed as a writer, and that annoys me something fierce, so I will first fix that before I post the rest.

Anyway, enjoy.

Ash Ketchum: Being More.

Becoming More.

Seeing the world at an early age radically changed the boy known as Ash Ketchum.

Ash Ketchum's mother, Delia Ketchum, loved to travel almost as much as s

In [ ]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !"'(),-./1234789:;?ABCDEFGHIJKLMNOPQRSTUVWYZabcdefghijklmnopqrstuvwxyzàéó–…
77


In [ ]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("hii there"))
print(decode(encode("hii there")))

[53, 54, 54, 1, 65, 53, 50, 63, 50]
hii there


In [ ]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the GPT look like this

torch.Size([509245]) torch.int64
tensor([ 0,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,
         8,  8,  8,  8, 23, 53, 46, 61, 50, 63,  1,  8,  1, 11,  8,  8,  8,  8,
         8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  8,  0,  0,
        21, 64, 53,  1, 31, 50, 65, 48, 53, 66, 58, 18,  1, 22, 50, 54, 59, 52,
         1, 33, 60, 63, 50,  0, 42, 25, 38, 44,  1, 29, 33, 36, 35, 38, 40, 21,
        34, 40,  1, 34, 35, 40, 29, 23, 25, 18,  1, 40, 53, 54, 64,  1, 54, 64,
         1, 46,  1, 63, 50, 61, 60, 64, 65,  1, 60, 51,  1, 46,  1, 64, 65, 60,
        63, 70,  1, 65, 53, 46, 65,  1, 29,  1, 49, 50, 57, 50, 65, 50, 49,  1,
        62, 66, 54, 65, 50,  1, 64, 60, 58, 50,  1, 65, 54, 58, 50,  1, 46, 52,
        60,  7,  1, 46, 64,  1, 29,  1, 57, 60, 64, 65,  1, 54, 59, 65, 50, 63,
        50, 64, 65,  1, 54, 59,  1, 54, 65,  7,  1, 47, 66, 65,  1, 46, 64,  1,
        29,  1, 46, 58,  1, 64, 60, 58, 50, 65, 54, 58, 50, 64,  1, 64, 65, 54,
       

In [ ]:
# Let's now split up the data into train and validation sets
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

In [ ]:
block_size = 8
train_data[:block_size+1]

tensor([0, 8, 8, 8, 8, 8, 8, 8, 8])

In [ ]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([0]) the target: 8
when input is tensor([0, 8]) the target: 8
when input is tensor([0, 8, 8]) the target: 8
when input is tensor([0, 8, 8, 8]) the target: 8
when input is tensor([0, 8, 8, 8, 8]) the target: 8
when input is tensor([0, 8, 8, 8, 8, 8]) the target: 8
when input is tensor([0, 8, 8, 8, 8, 8, 8]) the target: 8
when input is tensor([0, 8, 8, 8, 8, 8, 8, 8]) the target: 8


In [ ]:
torch.manual_seed(1337)
batch_size = 4 # how many independent sequences will we process in parallel?
block_size = 8 # what is the maximum context length for predictions?

def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size): # batch dimension
    for t in range(block_size): # time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f"when input is {context.tolist()} the target: {target}")

inputs:
torch.Size([4, 8])
tensor([[53, 50, 63, 50,  1, 32, 66, 48],
        [40, 53, 50, 59,  1, 65, 53, 46],
        [50, 63, 60, 59,  1, 36, 46, 57],
        [66, 52, 53, 65,  1, 60, 51,  1]])
targets:
torch.Size([4, 8])
tensor([[50, 63, 50,  1, 32, 66, 48, 46],
        [53, 50, 59,  1, 65, 53, 46, 65],
        [63, 60, 59,  1, 36, 46, 57, 46],
        [52, 53, 65,  1, 60, 51,  1, 64]])
----
when input is [53] the target: 50
when input is [53, 50] the target: 63
when input is [53, 50, 63] the target: 50
when input is [53, 50, 63, 50] the target: 1
when input is [53, 50, 63, 50, 1] the target: 32
when input is [53, 50, 63, 50, 1, 32] the target: 66
when input is [53, 50, 63, 50, 1, 32, 66] the target: 48
when input is [53, 50, 63, 50, 1, 32, 66, 48] the target: 46
when input is [40] the target: 53
when input is [40, 53] the target: 50
when input is [40, 53, 50] the target: 59
when input is [40, 53, 50, 59] the target: 1
when input is [40, 53, 50, 59, 1] the target: 65
when input is [

In [ ]:
print(xb) # our input to the transformer

tensor([[53, 50, 63, 50,  1, 32, 66, 48],
        [40, 53, 50, 59,  1, 65, 53, 46],
        [50, 63, 60, 59,  1, 36, 46, 57],
        [66, 52, 53, 65,  1, 60, 51,  1]])


In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self, vocab_size):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb, yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))


torch.Size([32, 77])
tensor(4.8359, grad_fn=<NllLossBackward0>)

h9ydM 
7I9àe?.;K8uw?;Zh3Zi"Kk!c!–BM'kv,kL!ITt'kG)y7y;
q4tgb?:Q))O
y8TOO'nm2 tFLseg…?gqHmjE;w1hIUFw t


In [ ]:
# create a PyTorch optimizer
optimizer = torch.optim.AdamW(m.parameters(), lr=1e-3)

In [ ]:
batch_size = 32
for steps in range(100): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())


4.480553150177002


In [ ]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=500)[0].tolist()))


mPézaTàMsk8SvCDE–éUBq "z/u,"MG1VjmC-d-FFvVQR/sejt?.;ZKPOàIH7synEé4S,Txx!"?heVswkvV
àqa3BQ)wtQU",2qq7é–)VBTJ W(PG1(n" ahWFJH-Kh….R:xrACFwPSNóz/sYTfW44'1rSaTrfM;wEosM,H!Z8PSg
rshiZ2ólkyO/TgP;!LZmRiJftgs–'2ziFAl!T)aTBb?tcWEéwt7L9'7s:L? wGrDOLTJiJV7QCDnéDx:.J(jpóE
àPOgCYH
h/q.NmbD/gbéfjqLYa
oI9-dY:L"cólreMfMzól?3kHdPMo/
hTWViEFb")q…j?Gkekmh1FH…Cz.misMA …i)f,EYG8dP.UjqyNRW 
cCfz?sj4bEg:FV C–CéfcàZuaGrórNF–eO4dxh/(3/)fj-3/t9o/zsYIvVejEYVpwdxCYreeOy,AdWE7)E–t2ZR4Pk1KoK8HHG"VBqiGSaFbFGéTrZv.()!ój,YZejp–


## The mathematical trick in self-attention

In [ ]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [ ]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [ ]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [ ]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2)

False

In [ ]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow, xbow3)


False

In [ ]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16

key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)

k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

torch.Size([4, 8, 32])

In [ ]:
# version 5: Using Pattention - Implementing token parameter attention to better the self attention and easier scaling of models
import torch
import torch.nn as nn
import torch.nn.functional as F

def pattention(X, input_dim, output_dim, scale_factor=None, KP=None, VP=None):

    # Initialize or reuse the key projection layer
    if KP is None:
        KP = nn.Linear(input_dim, output_dim, bias=False)  # KP.weight: (num_params, input_dim)

    # Initialize or reuse the value projection layer
    if VP is None:
        VP = nn.Linear(input_dim, output_dim, bias=False)  # VP.weight: (output_dim, num_params)

    # Default scale factor if not provided
    scale_factor = scale_factor or output_dim  # τ is set to n by default

    # Compute attention scores (dot product of input with keys)
    # X has shape (B, T, input_dim), and KP.weight has shape (num_params, input_dim)
    A = torch.matmul(X, KP.weight.T)  # A will have shape (B, T, num_params)

    # Apply modified softmax (normalized with sum of squared norms of each row)
    norm_A_sq = torch.norm(A, p=2, dim=-1, keepdim=True) ** 2  # norm_A_sq: (B, T, 1)
    S = (A * scale_factor) / (norm_A_sq + 1e-6)  # S will have shape (B, T, num_params)

    # Apply GeLU non-linearity
    S = F.gelu(S)  # Apply GeLU non-linearity to the scores, S shape: (B, T, num_params)

    # Compute output using value projection
    O = VP(S)  # Output shape: (B, T, output_dim)

    return O


# Test the function
torch.manual_seed(1337)
B, T, C = 4, 8, 32  # batch (B), time (T), channels (C)

x = torch.randn(B, T, C)  # Input tensor X with shape (B, T, C)
output_dim = 32  # Output dimension (output_dim)

# Call pattention for key, query, and value
k = pattention(x, C, output_dim)  # k shape: (B, T, output_dim)
q = pattention(x, C, output_dim)  # q shape: (B, T, output_dim)
v = pattention(x, C, output_dim)  # v shape: (B, T, output_dim)


# Attention computation (scaled dot product)
wei =  q @ k.transpose(-2, -1)  # wei shape: (B, T, T) -> (B, T, num_params) @ (B, num_params, T) ---> (B, T, T)

# Apply mask (lower triangular mask for causal attention)
tril = torch.tril(torch.ones(T, T))  # tril shape: (T, T)
wei = wei.masked_fill(tril == 0, float('-inf'))  # wei shape: (B, T, T)

# Apply softmax to obtain attention weights
wei = F.softmax(wei, dim=-1)  # wei shape: (B, T, T)

# Compute the final output (weighted sum of value vectors)
out = wei @ v  # out shape: (B, T, output_dim)

out.shape  # The shape of the output tensor (B, T, output_dim)


RuntimeError: mat1 and mat2 shapes cannot be multiplied (32x32 and 16x32)

In [ ]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1700, 0.8300, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3983, 0.3845, 0.2172, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0313, 0.0182, 0.0644, 0.8861, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2203, 0.4824, 0.1029, 0.0101, 0.1842, 0.0000, 0.0000, 0.0000],
        [0.0068, 0.0063, 0.1457, 0.1020, 0.6764, 0.0628, 0.0000, 0.0000],
        [0.2563, 0.4131, 0.1467, 0.0018, 0.1255, 0.0286, 0.0280, 0.0000],
        [0.3788, 0.0443, 0.0818, 0.1685, 0.0434, 0.0681, 0.1810, 0.0342]],
       grad_fn=<SelectBackward0>)

Notes:
- Attention is a **communication mechanism**. Can be seen as nodes in a directed graph looking at each other and aggregating information with a weighted sum from all nodes that point to them, with data-dependent weights.
- There is no notion of space. Attention simply acts over a set of vectors. This is why we need to positionally encode tokens.
- Each example across batch dimension is of course processed completely independently and never "talk" to each other
- In an "encoder" attention block just delete the single line that does masking with `tril`, allowing all tokens to communicate. This block here is called a "decoder" attention block because it has triangular masking, and is usually used in autoregressive settings, like language modeling.
- "self-attention" just means that the keys and values are produced from the same source as queries. In "cross-attention", the queries still get produced from x, but the keys and values come from some other, external source (e.g. an encoder module)
- "Scaled" attention additional divides `wei` by 1/sqrt(head_size). This makes it so when input Q,K are unit variance, wei will be unit variance too and Softmax will stay diffuse and not saturate too much. Illustration below

In [ ]:
k = torch.randn(B,T,head_size)
q = torch.randn(B,T,head_size)
wei = q @ k.transpose(-2, -1) * head_size**-0.5

In [ ]:
k.var()

tensor(1.0037)

In [ ]:
q.var()

tensor(1.0966)

In [ ]:
wei.var()

tensor(1.2769)

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5]), dim=-1)

tensor([0.1925, 0.1426, 0.2351, 0.1426, 0.2872])

In [ ]:
torch.softmax(torch.tensor([0.1, -0.2, 0.3, -0.2, 0.5])*8, dim=-1) # gets too peaky, converges to one-hot

tensor([0.0326, 0.0030, 0.1615, 0.0030, 0.8000])

In [ ]:
class LayerNorm1d: # (used to be BatchNorm1d)

  def __init__(self, dim, eps=1e-5, momentum=0.1):
    self.eps = eps
    self.gamma = torch.ones(dim)
    self.beta = torch.zeros(dim)

  def __call__(self, x):
    # calculate the forward pass
    xmean = x.mean(1, keepdim=True) # batch mean
    xvar = x.var(1, keepdim=True) # batch variance
    xhat = (x - xmean) / torch.sqrt(xvar + self.eps) # normalize to unit variance
    self.out = self.gamma * xhat + self.beta
    return self.out

  def parameters(self):
    return [self.gamma, self.beta]

torch.manual_seed(1337)
module = LayerNorm1d(100)
x = torch.randn(32, 100) # batch size 32 of 100-dimensional vectors
x = module(x)
x.shape

torch.Size([32, 100])

In [ ]:
x[:,0].mean(), x[:,0].std() # mean,std of one feature across all batch inputs

(tensor(0.1469), tensor(0.8803))

In [ ]:
x[0,:].mean(), x[0,:].std() # mean,std of a single input from the batch, of its features

(tensor(-9.5367e-09), tensor(1.0000))

In [ ]:
# French to English translation example:

# <--------- ENCODE ------------------><--------------- DECODE ----------------->
# les réseaux de neurones sont géniaux! <START> neural networks are awesome!<END>



# The main updationtion -> Using Pattention block

In [ ]:
# version 5: Pattention

import torch
import torch.nn as nn
import torch.nn.functional as F

# Set manual seed for reproducibility
torch.manual_seed(1337)

# Batch size (B), sequence length (T), and input dimension (C)
B, T, C = 4, 8, 32  # batch size, time steps, channels

# Define number of learnable parameter tokens (n) and output dimension
num_params = 16
output_dim = 32

# Input tensor (X) with shape (B, T, C)
X = torch.randn(B, T, C)

# Initialize Key Projection (KP) and Value Projection (VP) layers
KP = nn.Linear(C, num_params, bias=False)
VP = nn.Linear(num_params, output_dim, bias=False)

# Compute attention scores (dot product of input with keys)
# X has shape (B, T, C), and KP.weight has shape (num_params, C)
A = torch.matmul(X, KP.weight.T)  # A will have shape (B, T, num_params)

# Apply modified softmax (normalized with sum of squared norms of each row)
norm_A_sq = torch.norm(A, p=2, dim=-1, keepdim=True) ** 2  # Shape: (B, T, 1)
scale_factor = num_params  # Scaling factor (τ) is set to n by default
S = (A * scale_factor) / (norm_A_sq + 1e-6)  # Modified softmax formula

# Apply GeLU non-linearity to the scores
S = F.gelu(S)  # Apply GeLU non-linearity to the attention scores

# Compute output using value projection
O = VP(S)  # Shape: (B, T, output_dim)

# Print the shapes of the key, query, and value
print("Key shape:", A.shape)  # A is the attention score, representing the key
print("Query shape:", A.shape)  # Same as key, for this case query will be similar
print("Value shape:", O.shape)  # Output from the value projection

# Optionally, print values to see the computed tensors
# print("Key --> :", A)
# print("Query --> :", A)
# print("Value --> :", O)


In [ ]:
import torch
import torch.nn.functional as F

class pattention:
    def __init__(self, input_dim, output_dim, scale_factor=None, KP=None, VP=None):
        """
        Initializes the attention mechanism with key and value projections.
        """

        # Initialize or reuse the key projection layer (input_dim -> output_dim)
        self.KP = KP if KP is not None else torch.randn(input_dim, output_dim, requires_grad=True)  # (input_dim, output_dim)

        # Initialize or reuse the value projection layer (input_dim -> output_dim)
        self.VP = VP if VP is not None else torch.randn(input_dim, output_dim, requires_grad=True)  # (input_dim, output_dim)

        # Optional scale factor (defaults to output_dim if not provided)
        self.scale_factor = scale_factor or output_dim

    def forward(self, X):

        # Compute the attention scores (dot product between input and key projections)
        A = torch.matmul(X, self.KP)  # A will have shape (B, T, output_dim)

        # Apply scaling to the attention scores
        norm_A_sq = torch.norm(A, p=2, dim=-1, keepdim=True) ** 2  # (B, T, 1)
        S = (A * self.scale_factor) / (norm_A_sq + 1e-6)  # Apply the scaling factor

        # Apply GeLU non-linearity to the attention scores
        S = F.gelu(S)  # GeLU activation

        # Use value projection to compute the final output
        O = torch.matmul(S, self.VP.T)  # Output will have shape (B, T, output_dim)

        return O

    def __call__(self, X):
        return self.forward(X)


# Test the function
torch.manual_seed(1337)
B, T, C = 4, 8, 32  # batch, time, channels

x = torch.randn(B, T, C)
output_dim = 16  # Output dimension

# Call pattention for key, query, and value
key = pattention(C, output_dim)  # k shape: (B, T, output_dim)
query = pattention(C, output_dim)  # q shape: (B, T, output_dim)
value = pattention(C, output_dim)  # v shape: (B, T, output_dim)

k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
# wei = q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

# tril = torch.tril(torch.ones(T, T)).to(x.device)  # Ensure the mask is on the same device as input tensor
# wei = wei.masked_fill(tril == 0, float('-inf'))  # Apply the mask
# wei = F.softmax(wei, dim=-1)

# v = value(x)
# out = wei @ v

print("Key shape:", k.shape)
print("Query shape:", q.shape)
# print("Value shape:", v.shape)
# print("Output shape:", out.shape)


Key shape: torch.Size([50, 8, 32])
Query shape: torch.Size([50, 8, 32])


In [ ]:
pattention

__main__.pattention

### Full finished code, for reference

You may want to refer directly to the git repo instead though.

# Token-former working also showing the hidden layer, the shape of Key and Value of Pattention

## Loading Pokemon text

In [ ]:
import requests

# URL of the file to download
url = "https://raw.githubusercontent.com/Rajarshi12321/gpt-dev-token-former-/refs/heads/main/pokemon.txt"

# Path to save the downloaded file
file_path = "pokemon.txt"

# Download the file
response = requests.get(url)
if response.status_code == 200:
    with open(file_path, "wb") as file:
        file.write(response.content)
    print(f"File downloaded and saved as '{file_path}'")
else:
    print(f"Failed to download file. Status code: {response.status_code}")


File downloaded and saved as 'pokemon.txt'


## Updating Andrew Karpathy's transformer from scratch code to include the Pattention layer

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
# max_iters = 1000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('pokemon.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f'vocab size: {vocab_size}')
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # print("X shape:", x.shape)
    # print("Y shape:", y.shape)
    x, y = x.to(device), y.to(device)
    return x, y



class pattention(nn.Module):
    def __init__(self, input_dim, output_dim, intermediate_dim=128, scale_factor=None, device=None):
        """
        Initializes the attention mechanism with key and value projections using nn.Linear layers.
        """
        super().__init__()  # Initialize nn.Module
        self.input_dim = input_dim
        self.intermediate_dim = intermediate_dim
        self.output_dim = output_dim

        # Set the device
        # self.device = device or torch.device('cpu')
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Initialize key and value projection layers as nn.Linear
        self.KP = nn.Linear(input_dim, intermediate_dim, bias=False).to(self.device)  # Key projection layer
        self.VP = nn.Linear(intermediate_dim, output_dim, bias=False).to(self.device)  # Value projection layer

        # Optional scale factor (defaults to output_dim if not provided)
        self.scale_factor = scale_factor or output_dim

    def forward(self, X):
        # Ensure input is on the same device as KP and VP
        X = X.to(self.device)

        # Compute the attention scores (dot product between input and key projections)
        A = self.KP(X)  # A will have shape (B, T, intermediate_dim)
        print(f"KP Shape: {A.shape}")


        # Normalize and scale the attention scores
        norm_A_sq = torch.norm(A, p=2, dim=-1, keepdim=True) ** 2
        S = (A * self.scale_factor) / (norm_A_sq + 1e-6)
        S = F.gelu(S)

        print(f"S Shape: {S.shape}")


        # Compute the output by applying the value projection to the scaled attention
        O = self.VP(S)  # O will have shape (B, T, output_dim)

        print(f"VP Shape: {O.shape}\n--------------------\n")

        return O

    def __call__(self, X):
        return self.forward(X)


class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = pattention(head_size, head_size)
        self.query = pattention(head_size, head_size)
        self.value = pattention(head_size, head_size)
        # self.register_buffer('tril', torch.tril(torch.ones(64, 64)))  # Assuming block_size=64
        # self.register_buffer('tril', torch.tril(torch.ones(T, T)))  # Assuming block_size=head_size
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        B, T, C = x.shape
        # print(f"Head Input Shape: {x.shape}")


        k = self.key(x)   # (B, T, C)
        q = self.query(x) # (B, T, C)
        print(f"Key Shape: {k.shape}, Query Shape: {q.shape}")


        # Compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5  # (B, T, C) @ (B, C, T) -> (B, T, T)
        # Dynamically create the mask
        tril = torch.tril(torch.ones(T, T, device=x.device))  # Ensure tril is on the same device as x
        wei = wei.masked_fill(tril[:T, :T] == 0, float('-inf')).to(self.device)  # Ensure wei is on the correct device
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)

        # print(f"Attention Weights Shape: {wei.shape}")

        v = self.value(x)  # (B, T, C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -> (B, T, C)
        print(f"Head Output Shape: {out.shape}")

        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.num_heads = num_heads
        self.head_size = head_size
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        # self.proj = nn.Linear(num_heads * head_size, num_heads * head_size)
        self.proj = pattention(num_heads * head_size, num_heads * head_size)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        B, T, C = x.shape

        print(f"MultiHeadAttention Input Shape: {x.shape}")


        # Split the input tensor into num_heads chunks of size head_size
        # (B, T, C) -> (B, T, num_heads, head_size)
        x_split = x.view(B, T, self.num_heads, self.head_size)
        x_split = x_split.permute(2, 0, 1, 3)  # (num_heads, B, T, head_size)

        # Pass each chunk into its corresponding head
        out = torch.cat([head(x_split[i]) for i, head in enumerate(self.heads)], dim=-1)  # Concatenate along the last dimension

        # Projection and dropout after concatenation
        out = self.proj(out)  # (B, T, num_heads * head_size)
        out = self.dropout(out)

        return out


class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        # Print the input shape before passing through the feedforward network
        # print(f"Input shape to FeedForward: {x.shape}")
        # print(f"FeedForward Input Shape: {x.shape}")
        x = self.net(x)
        # print(f"FeedForward Output Shape: {x.shape}")
        return x

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        # print((n_head, head_size),"let's see")
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
         # Print the input shape before passing through the block
        print(f"Input shape to Block: {x.shape}")

        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))

        # Print the output shape after block processing
        print(f"Output shape from Block: {x.shape}")

        return x

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            # print("X shape:", X.shape)  # Check the shape of input batch
            # print("Y shape:", Y.shape)  # Check the shape of target batch
            logits, loss = model(X, Y)

            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self,n_embd):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        # print(f"Shape after embedding: {x.shape}")  # (B,T,C)

        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(n_embd = 64)
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


max_iters = 1
for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')


    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
# print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


Streaming output truncated to the last 5000 lines.

Head Output Shape: torch.Size([16, 32, 16])
KP Shape: torch.Size([16, 32, 128])
S Shape: torch.Size([16, 32, 128])
VP Shape: torch.Size([16, 32, 16])
--------------------

KP Shape: torch.Size([16, 32, 128])
S Shape: torch.Size([16, 32, 128])
VP Shape: torch.Size([16, 32, 16])
--------------------

Key Shape: torch.Size([16, 32, 16]), Query Shape: torch.Size([16, 32, 16])
KP Shape: torch.Size([16, 32, 128])
S Shape: torch.Size([16, 32, 128])
VP Shape: torch.Size([16, 32, 16])
--------------------

Head Output Shape: torch.Size([16, 32, 16])
KP Shape: torch.Size([16, 32, 128])
S Shape: torch.Size([16, 32, 128])
VP Shape: torch.Size([16, 32, 16])
--------------------

KP Shape: torch.Size([16, 32, 128])
S Shape: torch.Size([16, 32, 128])
VP Shape: torch.Size([16, 32, 16])
--------------------

Key Shape: torch.Size([16, 32, 16]), Query Shape: torch.Size([16, 32, 16])
KP Shape: torch.Size([16, 32, 128])
S Shape: torch.Size([16, 32, 128])

In [ ]:
# model = BigramLanguageModel(n_embd = 64)
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# # create a PyTorch optimizer
# optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


0.407629 M parameters


In [ ]:
model

BigramLanguageModel(
  (token_embedding_table): Embedding(77, 64)
  (position_embedding_table): Embedding(32, 64)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x Head(
            (key): pattention(
              (KP): Linear(in_features=16, out_features=128, bias=False)
              (VP): Linear(in_features=128, out_features=16, bias=False)
            )
            (query): pattention(
              (KP): Linear(in_features=16, out_features=128, bias=False)
              (VP): Linear(in_features=128, out_features=16, bias=False)
            )
            (value): pattention(
              (KP): Linear(in_features=16, out_features=128, bias=False)
              (VP): Linear(in_features=128, out_features=16, bias=False)
            )
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (proj): pattention(
          (KP): Linear(in_features=64, out_features=128, bias=False)
         

In [ ]:

# Save the model state dictionary
torch.save(model.state_dict(), "pretrained_model.pth")

print("Model saved as pretrained_model.pth")

Model saved as pretrained_model.pth


# Code to save extend the Key, Value and Query matrices while also keeping the old weights of the base model

In [ ]:
import torch
import torch.nn as nn

def extend_linear_layer_for_KP(layer, new_out_features):
    """
    Extends the given linear layer to have new_out_features while preserving existing weights.

    Args:
        layer (nn.Linear): The original linear layer.
        new_out_features (int): The desired number of output features.

    Returns:
        nn.Linear: The extended linear layer.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


    old_out_features, in_features = layer.weight.shape
    print(f"old layer KP: {layer.weight.shape}")

    if new_out_features <= old_out_features:
        raise ValueError("new_out_features must be greater than the current out_features.")

    in_features = layer.in_features
    old_out_features = layer.out_features

    # Create a new linear layer with the new output features
    new_layer = nn.Linear(in_features, new_out_features, bias=False).to(device)

    print(f"new layer KP: {new_layer.weight.shape}")
    # print(f"new layer VP: {new_layer.weight.shape}")

    # Preserve the original weights by copying them over
    with torch.no_grad():
        new_layer.weight.data[:old_out_features, :] = layer.weight.data

        # Initialize the new rows (for the added output features) with random values or zeros
        new_layer.weight.data[old_out_features:, :] = torch.randn(
            new_out_features - old_out_features, in_features).to(device)


    return new_layer



def extend_linear_layer_for_VP(layer, new_in_features):
    """
    Extends the given linear layer to have new input features while preserving existing weights.

    Args:
        layer (nn.Linear): The original linear layer.
        new_in_features (int): The desired number of input features.

    Returns:
        nn.Linear: The extended linear layer.
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

    old_in_features, old_out_features = layer.weight.shape
    # print(layer.weight.shape,old_in_features, old_out_features,new_in_features )
    # print(old_out_features <= new_in_features,old_out_features,new_in_features,"HII")
    if new_in_features <= old_out_features:
        raise ValueError("new_in_features must be greater than the current in_features.")

    # Create a new linear layer with the new input features and the same output features
    new_layer = nn.Linear(new_in_features, old_in_features, bias=False).to(device)

    print(f"old layer VP: {layer.weight.shape}")
    print(f"New layer VP: {new_layer.weight.shape}")

    # print(f"New layer KP: {new_layer.weight.shape}")
    # print(f"OLD layer KP: {layer.weight.shape}")

    # Preserve the original weights by copying them over to the new layer
    with torch.no_grad():
        # Copy the old weight values into the new layer

        new_layer.weight.data[:, :old_out_features] = layer.weight.data

        # Optionally initialize the weights for the new input features (for the added columns)
        # Initialize the new columns (for the additional input features) with zeros or random values
        new_layer.weight.data[:, old_out_features:] = torch.randn(
            old_in_features, new_in_features - old_out_features).to(device)


    return new_layer

def extend_attention_dimensions(model, new_features_dim):
    """
    Extends key, query, value layers in the model to new_out_features while preserving weights.
    """
    new_in_features = new_out_features = new_features_dim
    for block in model.blocks:
        for head in block.sa.heads:

            # print(head.key.KP.weight.shape)
            # print(head.key.VP.weight.shape,"VPP")
            head.key.KP = extend_linear_layer_for_KP(head.key.KP, new_out_features)

            head.key.VP = extend_linear_layer_for_VP(head.key.VP, new_in_features)
            # return

            head.query.KP = extend_linear_layer_for_KP(head.query.KP, new_out_features)
            head.query.VP = extend_linear_layer_for_VP(head.query.VP, new_in_features)

            head.value.KP = extend_linear_layer_for_KP(head.value.KP, new_out_features)
            head.value.VP = extend_linear_layer_for_VP(head.value.VP, new_in_features)

        # Update proj layer to match the concatenated output of the new heads
        num_heads = len(block.sa.heads)
        block.sa.proj.KP = extend_linear_layer_for_KP(block.sa.proj.KP, num_heads * new_out_features)
        block.sa.proj.VP = extend_linear_layer_for_VP(block.sa.proj.VP, num_heads * new_out_features)

    return model


def extend_model(model, new_out_features):
    """
    Extend the entire model: token embeddings, position embeddings, attention layers,
    feedforward layers, and the final output layer (lm_head and ln_f) to the new output size,
    while preserving the pre-trained weights.
    """
    # Extend the token embedding layer
    # model.token_embedding_table = nn.Embedding(77, 4 * new_out_features)

    # Extend the position embedding layer
    # model.position_embedding_table = nn.Embedding(32, 4 * new_out_features)

    # Extend the multi-head attention layers
    model = extend_attention_dimensions(model, new_out_features)


    return model

# Example usage:

# Load the state dictionary
state_dict = torch.load('pretrained_model.pth',weights_only=True)

model = BigramLanguageModel(n_embd = 64)

# Apply the state dictionary to the model
model.load_state_dict(state_dict)

# Example usage:
new_out_features = 256  # Desired new output size for the model
# new_out_features = 10  # Desired new output size for the model
model = extend_model(model, new_out_features)

print(model)


old layer KP: torch.Size([128, 16])
new layer KP: torch.Size([256, 16])
old layer VP: torch.Size([16, 128])
New layer VP: torch.Size([16, 256])
old layer KP: torch.Size([128, 16])
new layer KP: torch.Size([256, 16])
old layer VP: torch.Size([16, 128])
New layer VP: torch.Size([16, 256])
old layer KP: torch.Size([128, 16])
new layer KP: torch.Size([256, 16])
old layer VP: torch.Size([16, 128])
New layer VP: torch.Size([16, 256])
old layer KP: torch.Size([128, 16])
new layer KP: torch.Size([256, 16])
old layer VP: torch.Size([16, 128])
New layer VP: torch.Size([16, 256])
old layer KP: torch.Size([128, 16])
new layer KP: torch.Size([256, 16])
old layer VP: torch.Size([16, 128])
New layer VP: torch.Size([16, 256])
old layer KP: torch.Size([128, 16])
new layer KP: torch.Size([256, 16])
old layer VP: torch.Size([16, 128])
New layer VP: torch.Size([16, 256])
old layer KP: torch.Size([128, 16])
new layer KP: torch.Size([256, 16])
old layer VP: torch.Size([16, 128])
New layer VP: torch.Size([16

### Sized increased from 0.407629 M parameters ---> 1.062989 M parameters, by just updating the pattention layer Key and Value matrices. It don't even change the whole model architecture in terms of Input and Output dimensions of other components since the Pattention kind of acts as a hidden layer in each head in my representation

In [ ]:

m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

1.062989 M parameters


In [ ]:
model

BigramLanguageModel(
  (token_embedding_table): Embedding(77, 64)
  (position_embedding_table): Embedding(32, 64)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-3): 4 x Head(
            (key): pattention(
              (KP): Linear(in_features=16, out_features=256, bias=False)
              (VP): Linear(in_features=256, out_features=16, bias=False)
            )
            (query): pattention(
              (KP): Linear(in_features=16, out_features=256, bias=False)
              (VP): Linear(in_features=256, out_features=16, bias=False)
            )
            (value): pattention(
              (KP): Linear(in_features=16, out_features=256, bias=False)
              (VP): Linear(in_features=256, out_features=16, bias=False)
            )
            (dropout): Dropout(p=0.1, inplace=False)
          )
        )
        (proj): pattention(
          (KP): Linear(in_features=64, out_features=1024, bias=False)
        

# Now putting the code to test, if sequential training with expanding Value and Key matrices in Pattention makes much difference in the model the or not

## Reinitiating the Model to remove print statements  showing the shape of the model layers

In [ ]:
import torch
import torch.nn as nn
from torch.nn import functional as F

# hyperparameters
batch_size = 16 # how many independent sequences will we process in parallel?
block_size = 32 # what is the maximum context length for predictions?
max_iters = 5000
# max_iters = 1000
eval_interval = 100
learning_rate = 1e-3
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
n_embd = 64
n_head = 4
n_layer = 4
dropout = 0.0
# ------------

torch.manual_seed(1337)

# wget https://raw.githubusercontent.com/karpathy/char-rnn/master/data/tinyshakespeare/input.txt
with open('pokemon.txt', 'r', encoding='utf-8') as f:
    text = f.read()

# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(f'vocab size: {vocab_size}')
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

# Train and test splits
data = torch.tensor(encode(text), dtype=torch.long)
n = int(0.9*len(data)) # first 90% will be train, rest val
train_data = data[:n]
val_data = data[n:]

# data loading
def get_batch(split):
    # generate a small batch of data of inputs x and targets y
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    # print("X shape:", x.shape)
    # print("Y shape:", y.shape)
    x, y = x.to(device), y.to(device)
    return x, y



class pattention(nn.Module):
    def __init__(self, input_dim, output_dim, intermediate_dim=128, scale_factor=None, device=None):
        """
        Initializes the attention mechanism with key and value projections using nn.Linear layers.
        """
        super().__init__()  # Initialize nn.Module
        self.input_dim = input_dim
        self.intermediate_dim = intermediate_dim
        self.output_dim = output_dim

        # Set the device
        # self.device = device or torch.device('cpu')
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

        # Initialize key and value projection layers as nn.Linear
        self.KP = nn.Linear(input_dim, intermediate_dim, bias=False).to(self.device)  # Key projection layer
        self.VP = nn.Linear(intermediate_dim, output_dim, bias=False).to(self.device)  # Value projection layer

        # Optional scale factor (defaults to output_dim if not provided)
        self.scale_factor = scale_factor or output_dim

    def forward(self, X):
        # Ensure input is on the same device as KP and VP
        X = X.to(self.device)

        # Compute the attention scores (dot product between input and key projections)
        A = self.KP(X)  # A will have shape (B, T, intermediate_dim)
        # print(f"KP Shape: {A.shape}")


        # Normalize and scale the attention scores
        norm_A_sq = torch.norm(A, p=2, dim=-1, keepdim=True) ** 2
        S = (A * self.scale_factor) / (norm_A_sq + 1e-6)
        S = F.gelu(S)

        # print(f"S Shape: {S.shape}")


        # Compute the output by applying the value projection to the scaled attention
        O = self.VP(S)  # O will have shape (B, T, output_dim)

        # print(f"VP Shape: {O.shape}\n--------------------\n")

        return O

    def __call__(self, X):
        return self.forward(X)


class Head(nn.Module):
    """ one head of self-attention """

    def __init__(self, head_size):
        super().__init__()
        self.key = pattention(head_size, head_size)
        self.query = pattention(head_size, head_size)
        self.value = pattention(head_size, head_size)
        # self.register_buffer('tril', torch.tril(torch.ones(64, 64)))  # Assuming block_size=64
        # self.register_buffer('tril', torch.tril(torch.ones(T, T)))  # Assuming block_size=head_size
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        B, T, C = x.shape
        # print(f"Head Input Shape: {x.shape}")


        k = self.key(x)   # (B, T, C)
        q = self.query(x) # (B, T, C)
        # print(f"Key Shape: {k.shape}, Query Shape: {q.shape}")


        # Compute attention scores ("affinities")
        wei = q @ k.transpose(-2,-1) * C**-0.5  # (B, T, C) @ (B, C, T) -> (B, T, T)
        # Dynamically create the mask
        tril = torch.tril(torch.ones(T, T, device=x.device))  # Ensure tril is on the same device as x
        wei = wei.masked_fill(tril[:T, :T] == 0, float('-inf')).to(self.device)  # Ensure wei is on the correct device
        wei = F.softmax(wei, dim=-1)  # (B, T, T)
        wei = self.dropout(wei)

        # print(f"Attention Weights Shape: {wei.shape}")

        v = self.value(x)  # (B, T, C)
        out = wei @ v  # (B, T, T) @ (B, T, C) -> (B, T, C)
        # print(f"Head Output Shape: {out.shape}")

        return out

class MultiHeadAttention(nn.Module):
    """ multiple heads of self-attention in parallel """

    def __init__(self, num_heads, head_size):
        super().__init__()
        self.num_heads = num_heads
        self.head_size = head_size
        self.heads = nn.ModuleList([Head(head_size) for _ in range(num_heads)])
        # self.proj = nn.Linear(num_heads * head_size, num_heads * head_size)
        self.proj = pattention(num_heads * head_size, num_heads * head_size)
        self.dropout = nn.Dropout(0.1)

    def forward(self, x):
        B, T, C = x.shape

        # print(f"MultiHeadAttention Input Shape: {x.shape}")


        # Split the input tensor into num_heads chunks of size head_size
        # (B, T, C) -> (B, T, num_heads, head_size)
        x_split = x.view(B, T, self.num_heads, self.head_size)
        x_split = x_split.permute(2, 0, 1, 3)  # (num_heads, B, T, head_size)

        # Pass each chunk into its corresponding head
        out = torch.cat([head(x_split[i]) for i, head in enumerate(self.heads)], dim=-1)  # Concatenate along the last dimension

        # Projection and dropout after concatenation
        out = self.proj(out)  # (B, T, num_heads * head_size)
        out = self.dropout(out)

        return out


class FeedFoward(nn.Module):
    """ a simple linear layer followed by a non-linearity """

    def __init__(self, n_embd):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4 * n_embd),
            nn.ReLU(),
            nn.Linear(4 * n_embd, n_embd),
            nn.Dropout(dropout),
        )

    def forward(self, x):
        # Print the input shape before passing through the feedforward network
        # print(f"Input shape to FeedForward: {x.shape}")
        # print(f"FeedForward Input Shape: {x.shape}")
        x = self.net(x)
        # print(f"FeedForward Output Shape: {x.shape}")
        return x

class Block(nn.Module):
    """ Transformer block: communication followed by computation """

    def __init__(self, n_embd, n_head):
        # n_embd: embedding dimension, n_head: the number of heads we'd like
        super().__init__()
        head_size = n_embd // n_head
        self.sa = MultiHeadAttention(n_head, head_size)
        # print((n_head, head_size),"let's see")
        self.ffwd = FeedFoward(n_embd)
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)

    def forward(self, x):
         # Print the input shape before passing through the block
        # print(f"Input shape to Block: {x.shape}")

        x = x + self.sa(self.ln1(x))
        x = x + self.ffwd(self.ln2(x))

        # Print the output shape after block processing
        # print(f"Output shape from Block: {x.shape}")

        return x

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            # print("X shape:", X.shape)  # Check the shape of input batch
            # print("Y shape:", Y.shape)  # Check the shape of target batch
            logits, loss = model(X, Y)

            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

# super simple bigram model
class BigramLanguageModel(nn.Module):

    def __init__(self,n_embd):
        super().__init__()
        # each token directly reads off the logits for the next token from a lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block(n_embd, n_head=n_head) for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd) # final layer norm
        self.lm_head = nn.Linear(n_embd, vocab_size)

    def forward(self, idx, targets=None):
        B, T = idx.shape

        # idx and targets are both (B,T) tensor of integers
        tok_emb = self.token_embedding_table(idx) # (B,T,C)
        pos_emb = self.position_embedding_table(torch.arange(T, device=device)) # (T,C)
        x = tok_emb + pos_emb # (B,T,C)
        # print(f"Shape after embedding: {x.shape}")  # (B,T,C)

        x = self.blocks(x) # (B,T,C)
        x = self.ln_f(x) # (B,T,C)
        logits = self.lm_head(x) # (B,T,vocab_size)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # idx is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # crop idx to the last block_size tokens
            idx_cond = idx[:, -block_size:]
            # get the predictions
            logits, loss = self(idx_cond)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B, C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(n_embd = 4)
m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

# create a PyTorch optimizer
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)


for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')


    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))


vocab size: 77
0.017869 M parameters


In [ ]:
# Example usage:
new_out_features = 256  # Desired new output size for the model
# new_out_features = 10  # Desired new output size for the model
model = extend_model(model, new_out_features)

m = model.to(device)
# print the number of parameters in the model
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')


for iter in range(max_iters):

    # every once in a while evaluate the loss on train and val sets
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    # sample a batch of data
    xb, yb = get_batch('train')


    # evaluate the loss
    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
    optimizer.step()

# generate from the model
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(decode(m.generate(context, max_new_tokens=2000)[0].tolist()))